# Audio Synthetic Detection with Veridex

This notebook demonstrates how to use Veridex to detect AI-generated (deepfake) audio.

We will cover:
1. **Spectral Analysis**: Examining frequency domain artifacts.
2. **Wav2Vec**: Using foundation models for detection.

In [ ]:
# Install dependencies
!pip install veridex[audio]

In [ ]:
from veridex.audio import SpectralSignal, Wav2VecSignal
import numpy as np
import soundfile as sf
import os

## 1. Create/Load Sample Audio

We will generate a simple sine wave (as a dummy "synthetic" signal) and noise (as "natural" signal) for demonstration purposes if no file is provided.

In [ ]:
def create_dummy_audio(filename, type='sine', duration=3.0, sr=16000):
    t = np.linspace(0, duration, int(sr * duration))
    if type == 'sine':
        # Artificial pure tone
        audio = 0.5 * np.sin(2 * np.pi * 440 * t)
    else:
        # Random noise (more 'complex' spectrum)
        audio = 0.5 * np.random.normal(0, 1, len(t))
    
    sf.write(filename, audio, sr)
    return filename

sine_path = "dummy_sine.wav"
noise_path = "dummy_noise.wav"

create_dummy_audio(sine_path, 'sine')
create_dummy_audio(noise_path, 'noise')

## 2. Spectral Signal

Analyzes spectral features like rolloff, centroid, and bandwidth.

In [ ]:
spec_detector = SpectralSignal()

# A pure sine wave is very "unnatural" compared to speech
res_sine = spec_detector.detect(sine_path)
res_noise = spec_detector.detect(noise_path)

print(f"Sine Wave Score: {res_sine.score:.4f}")
print(f"Noise Score:     {res_noise.score:.4f}")

## 3. Wav2Vec Signal

Uses a pre-trained Wav2Vec 2.0 model to extract embeddings. 

**Note:** Downloading the model may take time.

In [ ]:
try:
    w2v_detector = Wav2VecSignal()
    
    res_w2v = w2v_detector.detect(noise_path)
    
    print(f"Wav2Vec Score: {res_w2v.score:.4f}")
    print(f"Confidence:    {res_w2v.confidence:.4f}")

except ImportError:
    print("Dependencies missing. Install via `pip install veridex[audio]`")
except Exception as e:
    print(f"Wav2Vec detection failed: {e}")